In [30]:
import pandas as pd
import numpy as np
import gurobi as gp
from gurobipy import GRB
import scipy as sp

In [2]:
df = pd.read_csv('retail_price.csv')

In [18]:
df['month'] = pd.to_datetime(df['month_year'], dayfirst=True).dt.month
df['year'] = pd.to_datetime(df['month_year'], dayfirst = True).dt.year

In [19]:
df[df['product_category_name'] == 'bed_bath_table'][df['month'] == 5][df['year'] == 2017]

/tmp/ipykernel_29115/2229989431.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['product_category_name'] == 'bed_bath_table'][df['month'] == 5][df['year'] == 2017]


,product_id,product_category_name,month_year,qty,total_price,freight_price,unit_price,product_name_lenght,product_description_lenght,product_photos_qty,...,comp_1,ps1,fp1,comp_2,ps2,fp2,comp_3,ps3,fp3,lag_price
0,bed1,bed_bath_table,01-05-2017,1,45.95,15.100000,45.95,39,161,2,...,89.9,3.9,15.011897,215.0,4.4,8.76,45.95,4.0,15.1,45.90
441,bed2,bed_bath_table,01-05-2017,58,5214.20,15.011897,89.90,54,245,1,...,89.9,3.9,15.011897,215.0,4.4,8.76,45.95,4.0,15.1,89.90
671,bed5,bed_bath_table,01-05-2017,1,215.00,8.760000,215.00,56,162,5,...,89.9,3.9,15.011897,215.0,4.4,8.76,45.95,4.0,15.1,214.95


In [29]:
n_customers = 1000
inventory_space = 1000
selling = np.array([45.95,
89.9,
99,
47.9,
])

buying = selling - np.random.uniform(10,20)
probabilities = np.exp(-selling / 100) / np.sum(np.exp(-selling/100))
I = range(len(buying))

In [25]:
probabilities

array([0.31120092, 0.20052496, 0.18308284, 0.30519128])

In [34]:
m = gp.Model()
stock = m.addVars(len(I), vtype = GRB.INTEGER)
is_chosen = m.addVars(len(I), vtype = GRB.BINARY)
sold = m.addVars(len(I), vtype = GRB.INTEGER)

m.setObjective(
    gp.quicksum(sold[i]* selling[i] - stock[i]*buying[i] for i in I),
    GRB.MAXIMIZE
)
m.addConstr(gp.quicksum(stock[i] for i in range(4)) <= inventory_space)
m.addConstrs(stock[i] <= is_chosen[i] * n_customers for i in I)
m.addConstrs(sold[i] <= stock[i] for i in I)
m.addConstrs(sold[i] <= n_customers * probabilities[i]  * is_chosen[i] for i in I)

m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 12 columns and 28 nonzeros
Model fingerprint: 0x4c155f48
Variable types: 0 continuous, 12 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 17587.7 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.758766265665e+04, best bound 1.758766265665e+04, gap 0.0000%


In [35]:
stock

{0: <gurobi.Var C0 (value 311.0)>,
 1: <gurobi.Var C1 (value 200.0)>,
 2: <gurobi.Var C2 (value 183.0)>,
 3: <gurobi.Var C3 (value 305.0)>}

In [36]:
buying

array([28.34473208, 72.29473208, 81.39473208, 30.29473208])

In [37]:
selling

array([45.95, 89.9 , 99.  , 47.9 ])

In [44]:
n_customers = 1000
inventory_space = 1000
selling = np.array([45.95,
89.9,
99,
47.9,
])


rating = np.array([3.8, 4.5, 4.8,4.0])
MAX_RATING = 5.

buying = selling - np.random.uniform(10,20)
p1 = np.exp(-selling / 100) / np.sum(np.exp(-selling/100))
p2 = np.exp(-selling*(MAX_RATING - rating) / 100) / np.sum(np.exp(-selling*(MAX_RATING - rating)/100))
I = range(len(buying))

In [45]:
p1

array([0.31120092, 0.20052496, 0.18308284, 0.30519128])

In [46]:
p2

array([0.21709593, 0.24038441, 0.30912302, 0.23339664])

In [47]:
m = gp.Model()
stock = m.addVars(len(I), vtype = GRB.INTEGER)
is_chosen = m.addVars(len(I), vtype = GRB.BINARY)
sold = m.addVars(len(I), vtype = GRB.INTEGER)

m.setObjective(
    gp.quicksum(sold[i]* selling[i] - stock[i]*buying[i] for i in I),
    GRB.MAXIMIZE
)
m.addConstr(gp.quicksum(stock[i] for i in range(4)) <= inventory_space)
m.addConstrs(stock[i] <= is_chosen[i] * n_customers for i in I)
m.addConstrs(sold[i] <= stock[i] for i in I)
m.addConstrs(sold[i] <= n_customers * p2[i]  * is_chosen[i] for i in I)

m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 12 columns and 28 nonzeros
Model fingerprint: 0xbbe1fd89
Variable types: 0 continuous, 12 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 13312.2 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.331216606617e+04, best bound 1.331216606617e+04, gap 0.0000%


In [48]:
stock

{0: <gurobi.Var C0 (value 217.0)>,
 1: <gurobi.Var C1 (value 240.0)>,
 2: <gurobi.Var C2 (value 309.0)>,
 3: <gurobi.Var C3 (value 233.0)>}

# competitors

In [61]:
comp = np.array(
    [
    [74,74,74,77.9,89.9],
    [39.24,74,84.9,44.2,163.4]
]
)
r = np.array([
    [3.9,3.9,3.9,3.9,3.9],
    [4.4,4.4,3.3,4.2,4.2]
])

c_score = r / comp
c_score = np.mean(c_score, axis =1)

In [57]:
score

array([0.05031077, 0.06623712])

In [59]:
price = np.array([39.24,74,84.9,44.15,163.4])
ratings = np.array([4,3.9,3.3,4.2,4.4])
score = np.mean(ratings / price)
score

0.06311335644821126

In [64]:
scores = [0.05031077, 0.06623712, score]

In [65]:
np.exp(scores) / np.sum(np.exp(scores))

array([0.33014864, 0.33544879, 0.33440257])